In [1]:
import geopandas as gpd
import numpy as np

import sys
sys.path.append('../Mask_RCNN')
import mrcnn.model as modellib

# Autoria
import canopy_detection as orca
import hlb_utils

In [2]:
logs = '../Mask_RCNN/logs'
config = orca.OrangeCanopyConfig()
dataset = '../datasets/segmentacao_todos/'

## Treinamento

In [3]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=logs)

In [6]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20211223T2344/mask_rcnn_oranges_trees_canopy_0029.h5'

model.load_weights(weights, by_name=True)

Re-starting from epoch 29


In [ ]:
weights = '../Mask_RCNN/mask_rcnn_coco.h5'

model.load_weights(weights, by_name=True, exclude=["mrcnn_class_logits", 
                                                   "mrcnn_bbox_fc",  
                                                   "mrcnn_bbox", 
                                                   "mrcnn_mask", 
                                                   #"conv1"
                                                  ])

In [7]:
orca.train(model, config, 'all_4', dataset=dataset)

Training Heads...

Starting at epoch 29. LR=0.001

Checkpoint Path: ../Mask_RCNN/logs\oranges_trees_canopy20211223T2344\mask_rcnn_oranges_trees_canopy_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (

Training All Two...

Starting at epoch 29. LR=0.0001

Checkpoint Path: ../Mask_RCNN/logs\oranges_trees_canopy20211223T2344\mask_rcnn_oranges_trees_canopy_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
r

Epoch 30/39


C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\framework\indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\framework\indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("train

1031/1031 [==============================] - ETA: 0s - batch: 515.0000 - size: 1.0000 - loss: 13.2193 - rpn_class_loss: 0.1482 - rpn_bbox_loss: 2.9065 - mrcnn_class_loss: 0.0115 - mrcnn_bbox_loss: 0.0694 - mrcnn_mask_loss: 0.1691

...

C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1031/1031 [==============================] - 913s 866ms/step - batch: 515.0000 - size: 1.0000 - loss: 17.3504 - rpn_class_loss: 0.1402 - rpn_bbox_loss: 3.0855 - mrcnn_class_loss: 0.0121 - mrcnn_bbox_loss: 0.0660 - mrcnn_mask_loss: 0.1662 - val_loss: 15.3272 - val_rpn_class_loss: 0.1292 - val_rpn_bbox_loss: 2.6387 - val_mrcnn_class_loss: 0.0135 - val_mrcnn_bbox_loss: 0.0986 - val_mrcnn_mask_loss: 0.1854
Epoch 41/50
1031/1031 [==============================] - 973s 944ms/step - batch: 515.0000 - size: 1.0000 - loss: 19.9943 - rpn_class_loss: 0.1639 - rpn_bbox_loss: 3.5925 - mrcnn_class_loss: 0.0119 - mrcnn_bbox_loss: 0.0640 - mrcnn_mask_loss: 0.1665 - val_loss: 17.4194 - val_rpn_class_loss: 0.1299 - val_rpn_bbox_loss: 3.0710 - val_mrcnn_class_loss: 0.0107 - val_mrcnn_bbox_loss: 0.0919 - val_mrcnn_mask_loss: 0.1804
Epoch 42/50
1031/1031 [==============================] - 1062s 1s/step - batch: 515.0000 - size: 1.0000 - loss: 16.1002 - rpn_class_loss: 0.1350 - rpn_bbox_loss: 2.8467 - mrcnn

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-3be9da0b57d6>", line 1, in <module>
    orca.train(model, config, 'all_4', dataset=dataset)
  File "C:\Users\Colaborador\greening\PYTHON\canopy_detection.py", line 282, in train
    model.train(train_dataset, val_dataset, learning_rate=config.LEARNING_RATE/1000, epochs=50, augmentation=augmentation, layers='all')
  File "../Mask_RCNN\mrcnn\model.py", line 2367, in train
    use_multiprocessing=workers > 1,
  File "C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\keras\engine\training_v1.py", line 808, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\Colaborador\anaconda3\envs\rcnn\lib\site-packages\tensorflow\python\keras\engine\training_generator_v1.py", line 593, in fit
    steps_name='steps_per_epoch'

TypeError: object of type 'NoneType' has no len()

## Inferência

In [ ]:
model = modellib.MaskRCNN(mode='inference', config=config, model_dir=logs)

In [ ]:
weights = '../Mask_RCNN/logs/oranges_trees_canopy20211029T0315/mask_rcnn_oranges_trees_canopy_0060.h5'

model.load_weights(weights, by_name=True)

In [ ]:
import geopandas as gpd
img_test = gpd.read_file('../datasets/area_estudo_teste2-DSM/img_grid.geojson')
ids = list(img_test.loc[img_test['split_samples'] == 'test'].index)
ids

In [ ]:
results = orca.prediction(model, dataset, id_list=ids, verbose=0, first=False, join=True)

In [ ]:
results